In [8]:
# 初始化環境
from rag_to_vector_db_pinecone.rag_pipeline.chunks_embeding_2_vector_db import (
    jina_embedding,
)
from pprint import pprint

import json
from pinecone import Pinecone
from langchain_openai import ChatOpenAI

# 載入 env 設定
with open('env.json', 'r', encoding='utf-8') as file:
    config = json.load(file)

# 初始化 Pinecone gRPC 客戶端
pc = Pinecone(api_key=config['pinecone_api_key'])
index = pc.Index("rag-vector-db")

# 初始化 OpenAI 模型
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # 可以改為 gpt-3.5-turbo 測試
    openai_api_key=config["chat_gpt_key"],
    temperature=0.6
)

In [2]:
# 使用者輸入查詢 pinecone
def query_pincone(query, config):
    query_embedding = jina_embedding(query, config)
    results = index.query(
        vector=query_embedding,
        top_k=3,
        include_metadata=True
    )

    matches = results['matches']
    # 用 id 匹配資料庫文字
    with open('data/chunks.json', 'r', encoding='utf-8') as file:
        chunks = json.load(file)

    match_chunks_data = []
    for match in matches:
        for chunk in chunks:
            if chunk['id'] == match['id']:
                match_chunks_data.append({
                    "id" : match['id'],
                    "score" : match['score'],
                    "text" : chunk['text']
                })
                break

    return match_chunks_data


def create_augmented_prompt(query, match_chunks_data):
    # 將檢索到的上下文整合成一個字符串
    contexts = ""
    for i, context in enumerate(match_chunks_data):
        contexts += f"參考資料 {i+1} [相關度: {context['score']:.2f}]:\n{context['text']}\n\n"
    
    # 構建結構化提示
    prompt = f"""
    以下是使用者的查詢問題，請根據提供的參考資料回答。如果參考資料中沒有相關資訊，請明確說明。

    使用者查詢: {query}
    請根據以下參考資料提供準確、簡潔且有條理的回答。引用資料時請註明來源編號。

    {contexts}
    """
    
    return prompt


def query_llm(prompt):
    try:
        response = llm.invoke(prompt)
        return response.content
    except:
        return f"LLM 錯誤"

In [10]:
def rag_query(query):
    # 1. 尋找知識庫
    match_chunks_data = query_pincone(query, config)
    print('==========匹配向量資料庫=============')
    pprint(match_chunks_data, sort_dicts=False)

    # 2. 將 知識庫資料結合至 prompt
    prompt = create_augmented_prompt(query, match_chunks_data)
    
    # 3. prompt 引入 LLM
    rag_result = query_llm(prompt)
    
    # 4. rag query
    print('\n\n\n================ RAG result ==========')
    print(rag_result)

In [9]:
rag_query('最具成本效益的家用防霉方法是什麼？')

==========匹配向量資料庫=============
[{'id': 'chunk22',
  'score': 0.687544703,
  'text': '### 6.3 商業防霉產品評估\n'
          '市面上常見防霉產品的效果比較：\n'
          '- 抗菌噴霧：方便使用，適合預防性處理\n'
          '- 氯基漂白劑：殺菌效果好，但可能損傷某些材質\n'
          '- 專業皮革防霉劑：針對皮革設計，通常含防水成分\n'
          '- 天然防霉劑：如茶樹油，對敏感人群更友好'},
 {'id': 'chunk31',
  'score': 0.625386596,
  'text': '### 9.3 家庭防霉的經濟考量\n'
          '消費者層面的防霉經濟學：\n'
          '- 高端鞋類的保養投資與延長壽命收益\n'
          '- 防霉產品的家庭預算分配\n'
          '- 鞋類損失與更換成本分析\n'
          '\n'
          '## 第十章：行業標準與最佳實踐\n'
          '\n'
          '### 10.1 國際標準與規範\n'
          '與鞋類防霉相關的行業標準：\n'
          '- ISO防霉測試標準\n'
          '- 鞋類儲存環境標準\n'
          '- 防霉處理化學品安全規範\n'
          '- 產品防霉性能標準\n'
          '\n'
          '### 10.2 製造商防霉指南\n'
          '鞋類生產商的防霉實踐建議：\n'
          '- 原材料預處理\n'
          '- 生產過程中的防霉控制點\n'
          '- 產品包裝前的處理流程\n'
          '- 質量控制檢查項目'},
 {'id': 'chunk32',
  'score': 0.615278661,
  'text': '### 10.3 零售商與消費者防霉清單\n'
          '實用的防霉檢查清單：\n'
          '- 零售環境日常檢查項目